In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

2024-05-04 01:39:47.320433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 01:39:47.320605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 01:39:47.450754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data = pd.read_csv('../input/final-sarcasm-dataset/Dataset.csv')
print(data.columns)
data = data[['Emojis','Sarcasm']]
data.head()

Index(['Comments', 'Emojis', 'Text', 'Sarcasm'], dtype='object')


,Emojis,Sarcasm
0,😎,1
1,😂 😂 😂,1
2,😂 😂 😂 😂 😂 😂 😂,1
3,😄,1
4,😂,1


In [3]:
data['Emojis'] = data['Emojis'].astype(str)

print(data[ data['Sarcasm'] == 1].size)
print(data[ data['Sarcasm'] == 0].size)

data.head()

18520
26140


,Emojis,Sarcasm
0,😎,1
1,😂 😂 😂,1
2,😂 😂 😂 😂 😂 😂 😂,1
3,😄,1
4,😂,1


In [4]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' .,?/-!#&')
tokenizer.fit_on_texts(data['Emojis'].values)
X = tokenizer.texts_to_sequences(data['Emojis'].values)
X = pad_sequences(X)

In [5]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.build((None, X.shape[1]))
print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1, 128)         │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 1, 128)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 196)            │       254,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           394 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 511,194 (1.95 MB)

 Trainable params: 511,194 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
Y = pd.get_dummies(data['Sarcasm']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14961, 1) (14961, 2)
(7369, 1) (7369, 2)


In [7]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
468/468 - 7s - 15ms/step - accuracy: 0.8249 - loss: 0.3843
Epoch 2/20
468/468 - 2s - 5ms/step - accuracy: 0.8579 - loss: 0.3261
Epoch 3/20
468/468 - 2s - 5ms/step - accuracy: 0.8642 - loss: 0.3076
Epoch 4/20
468/468 - 2s - 5ms/step - accuracy: 0.8640 - loss: 0.2996
Epoch 5/20
468/468 - 2s - 5ms/step - accuracy: 0.8634 - loss: 0.2964
Epoch 6/20
468/468 - 2s - 5ms/step - accuracy: 0.8649 - loss: 0.2951
Epoch 7/20
468/468 - 2s - 5ms/step - accuracy: 0.8630 - loss: 0.2948
Epoch 8/20
468/468 - 2s - 5ms/step - accuracy: 0.8653 - loss: 0.2917
Epoch 9/20
468/468 - 2s - 5ms/step - accuracy: 0.8652 - loss: 0.2914
Epoch 10/20
468/468 - 2s - 5ms/step - accuracy: 0.8652 - loss: 0.2905
Epoch 11/20
468/468 - 2s - 5ms/step - accuracy: 0.8665 - loss: 0.2893
Epoch 12/20
468/468 - 2s - 5ms/step - accuracy: 0.8657 - loss: 0.2887
Epoch 13/20
468/468 - 2s - 5ms/step - accuracy: 0.8660 - loss: 0.2886
Epoch 14/20
468/468 - 2s - 5ms/step - accuracy: 0.8680 - loss: 0.2870
Epoch 15/20
468/468 - 2s - 5

In [8]:
model.save("Models/LSTM_only_emojis.h5")

In [9]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

184/184 - 1s - 4ms/step - accuracy: 0.8531 - loss: 0.3649
score: 0.36
acc: 0.85


In [10]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose=0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 74.42244224422441 %
neg_acc 93.28859060402685 %


In [11]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 0s - 212ms/step
negative
